In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import Packages.ClusteringHelper as ch
import numpy as np
from tqdm import tqdm
from collections import Counter
import string
from nltk.corpus import stopwords

In [3]:
text, data = ch.read_aida_yago_conll(
    "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\AIDA-YAGO2-dataset.tsv")
save = False
if save:
    text_file = open('text.txt', 'w')
    text_file.write(text)
    text_file.close()
data

,documents,tokens,indexes,word_indexes,mentions,entities,wikidatas,numeric_codes,alpha_codes
0,0,EU,"(0, 2)",0,,,,,
1,0,rejects,"(3, 10)",1,,,,,
2,0,German,"(11, 17)",2,German,Germany,http://en.wikipedia.org/wiki/Germany,11867,/m/0345h
3,0,call,"(18, 22)",3,,,,,
4,0,to,"(23, 25)",4,,,,,
...,...,...,...,...,...,...,...,...,...
285584,1392,younger,"(1342, 1349)",265,,,,,
285585,1392,brother,"(1350, 1357)",266,,,,,
285586,1392,",","(1358, 1359)",267,,,,,
285587,1392,Bobby,"(1360, 1365)",268,Bobby,Bobby_Charlton,http://en.wikipedia.org/wiki/Bobby_Charlton,4224,/m/01c8x


In [4]:
data = ch.filter_data(data, 3)
data

,documents,tokens,indexes,word_indexes,mentions,entities,wikidatas,numeric_codes,alpha_codes
0,0,EU,"(0, 2)",0,,,,,
1,0,rejects,"(3, 10)",1,,,,,
2,0,German,"(11, 17)",2,German,Germany,http://en.wikipedia.org/wiki/Germany,11867,/m/0345h
3,0,call,"(18, 22)",3,,,,,
4,0,to,"(23, 25)",4,,,,,
...,...,...,...,...,...,...,...,...,...
285584,1392,younger,"(1342, 1349)",265,,,,,
285585,1392,brother,"(1350, 1357)",266,,,,,
285586,1392,",","(1358, 1359)",267,,,,,
285587,1392,Bobby,"(1360, 1365)",268,,,http://en.wikipedia.org/wiki/Bobby_Charlton,4224,/m/01c8x


In [5]:
# Gold standard
ents_data = data[data['entities'] != '']
entities = ents_data['entities'].values
mentions = ents_data['tokens'].values
docs = ents_data['documents'].values
word_indexes = ents_data['word_indexes'].values
unique_entities = np.unique(entities)
gold_standard = Counter(entities)
gold_standard = dict(sorted(gold_standard.items(), key=lambda item: item[1], reverse=True))

#### Gensim word2vec

In [6]:
lowercase = True
remove_stopwords = True
remove_numbers = True
remove_symbols = True

text_preprocessed = text
stop_words = set(stopwords.words("english"))

if remove_symbols:
    text_preprocessed = [x.translate(str.maketrans('', '', string.punctuation)) for x in text_preprocessed]

# rimuovo numeri e punteggiatura
if lowercase:
    text_splitted_training = [x.lower().split() for x in text_preprocessed]
    text_splitted = [x.lower().split() for x in text]
else:
    text_splitted_training = [x.split() for x in text_preprocessed]
    text_splitted = [x.split() for x in text]

if remove_numbers:
    text_splitted_training = [[y for y in x if not y.isnumeric()] for x in text_splitted_training]

if remove_stopwords:
    text_splitted_training = [[y for y in x if y.lower() not in stop_words] for x in text_splitted_training]

Model selection

In [7]:
import os
from gensim.models import KeyedVectors

model_path = "D:\Sgmon\Documents\Magistrale\TESI\gensim-data"

model = KeyedVectors.load_word2vec_format(
    os.path.join(model_path, 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-50.gz'), binary=False)

In [8]:
leven_cluster = np.loadtxt("../aida-yago2-dataset/db_cluster_levestein_0_3.txt", dtype=np.int32)
lev_cluster_dict = {}
windows = 30
doc_windows = 20
for i, x in enumerate(leven_cluster):
    try:
        lev_cluster_dict[x].append(
            (mentions[i], entities[i], docs[i], word_indexes[i],
             ch.get_context(text_splitted[docs[i]], word_indexes[i], windows=windows)))
    except:
        lev_cluster_dict[x] = [
            (mentions[i], entities[i], docs[i], word_indexes[i],
             ch.get_context(text_splitted[docs[i]], word_indexes[i], windows=windows))]
# """se uso un cluster diverso da minimo 0 allora scommento"""
# del lev_cluster_dict[-1]

In [9]:
cluster_all = []
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN

# from hdbscan import HDBSCAN

clusterizator = AgglomerativeClustering(n_clusters=None, affinity='cosine', distance_threshold=0.035, linkage="single")
# clusterizator = DBSCAN(min_samples=0, metric="cosine", n_jobs=-1, eps=0.1)
# clusterizator = HDBSCAN(min_cluster_size=2, metric="euclidean", min_samples=1, cluster_selection_epsilon=0.4)
vectors_all = []
for x in tqdm(lev_cluster_dict):
    contexts_vectorized = ch.calculate_context_vector(model, lev_cluster_dict[x], text_splitted, doc_window=doc_windows,
                                                      doc_context_alpha=0.3, word_context_aplha=.7)
    if len(contexts_vectorized) > 1:
        cluster = clusterizator.fit_predict(contexts_vectorized)
    else:
        cluster = np.zeros(1)
    cluster_all.append(cluster)
    vectors_all.append(contexts_vectorized)
    # print(cluster_all)
    # break

100%|██████████| 2750/2750 [00:04<00:00, 620.57it/s] 


In [10]:
lev_cluster_dict_vectorized = {}
for index in lev_cluster_dict.keys():
    lev_cluster_dict_vectorized[index] = []
    for i, element in enumerate(lev_cluster_dict[index]):
        # print((*element, vectors_all[index][i]))
        lev_cluster_dict_vectorized[index].append((*element, vectors_all[index][i]))

In [11]:
clustered_entities = [[(row[1], row[5]) for row in lev_cluster_dict_vectorized[index]] for index in lev_cluster_dict_vectorized.keys()]
len(clustered_entities)

2750

In [12]:
clusters_splitted = []
clusters_vectors = []
for n_cluster in range(len(clustered_entities)):
    cluster_unique = dict.fromkeys(set(cluster_all[n_cluster]), [])
    cluster_unique_vector = dict.fromkeys(set(cluster_all[n_cluster]), [])
    for key in cluster_unique:
        cluster_unique[key] = [clustered_entities[n_cluster][index][0] for index in
                               range(len(clustered_entities[n_cluster])) if cluster_all[n_cluster][index] == key]
        cluster_unique_vector[key] = [clustered_entities[n_cluster][index][1] for index in
                               range(len(clustered_entities[n_cluster])) if cluster_all[n_cluster][index] == key]
    clusters_vectors.append(cluster_unique_vector)
    clusters_splitted.append(cluster_unique)

In [13]:
clusters_splitted_vector_dict = [np.mean(x[y],axis=0)  for x in clusters_vectors for y in x if len(x[y]) > 1]

In [14]:
clusters_splitted_dict = [dict(Counter(x[y])) for x in clusters_splitted for y in x if len (x[y]) > 1]

In [15]:
clusterizator = AgglomerativeClustering(n_clusters=None, affinity='cosine', distance_threshold=0.015, linkage="single")

clusters_of_cluster = clusterizator.fit_predict(clusters_splitted_vector_dict)

In [16]:
coc_dict = {}
for index, el in enumerate(clusters_of_cluster):
    try:
        coc_dict[el] = coc_dict[el] + clusters_splitted_dict[index]
    except:
        coc_dict[el] = clusters_splitted_dict[index]
len(coc_dict.keys())

622

In [17]:
import Packages.ClusteringHelper as ch
golden_standard_dict = ch.get_gold_standard_dict(ents_data)

### B-CUBED

In [18]:
from tqdm import tqdm
# B-cubed - precision
bcubed_precision_num = 0
for gold_key in tqdm(golden_standard_dict.keys()):
    for lev_key in coc_dict.keys():
        try:
            bcubed_precision_num = bcubed_precision_num + (pow(coc_dict[lev_key][gold_key], 2) /
                                       sum(coc_dict[lev_key].values()))
        except:
            pass
bcubed_precision = bcubed_precision_num/sum([sum(x.values()) for x in coc_dict.values()])
bcubed_precision

100%|██████████| 2089/2089 [00:00<00:00, 5301.36it/s]


0.9444019389918704

In [19]:
# B-cubed - recall
bcubed_recall_num = 0
for gold_key in tqdm(golden_standard_dict.keys()):
    for lev_key in coc_dict.keys():
        try:
            bcubed_recall_num = bcubed_recall_num + (pow(coc_dict[lev_key][gold_key], 2) /
                                       golden_standard_dict[gold_key])
        except:
            pass
bcubed_recall = bcubed_recall_num/ents_data.shape[0]
bcubed_recall

100%|██████████| 2089/2089 [00:00<00:00, 5738.73it/s]


0.04932018777643609

In [20]:
bcubed_f1 = (2 * (bcubed_recall * bcubed_precision)) / (
        bcubed_precision + bcubed_recall)
bcubed_f1

0.09374467914684848

CEAFm precision

In [21]:
max_evaluation_dict = ch.get_optimal_alignment(coc_dict, set(entities))

In [25]:
max_evaluation_dict

{'Hanover': 2,
 'Hapoel_Tzafririm_Holon_F.C.': 2,
 'Miriam_Vogt': 4,
 'Jews': 2,
 'Jürgen_Klinsmann': 2,
 'Boutros_Boutros-Ghali': 2,
 'Colchester': 0,
 'Walter_Little_(rugby_player)': 0,
 'Portugal_national_football_team': 4,
 'European_Commission': 10,
 'Alan_Greenspan': 3,
 'Sebastien_Tortelli': 3,
 'KK_Partizan': 4,
 'Berlin': 3,
 'AOL': 0,
 'Dejan_Savićević': 0,
 'Susi_Susanti': 3,
 'Bertelsmann': 2,
 'Marcelo_Silva_Ramos': 0,
 'Germany': 5,
 'Bonn': 2,
 'Warsaw': 2,
 'Sven_Strüver': 0,
 'Amsterdam': 2,
 'Hamburg': 2,
 'San_José,_Costa_Rica': 3,
 'Bosnia': 4,
 'Newbury,_Berkshire': 3,
 'The_Times': 2,
 'Antony_Marlow': 2,
 'United_Kingdom': 2,
 'Luis_Oliveira': 0,
 'NATO': 2,
 'Belgium': 2,
 'Brussels': 3,
 'WA_Reds': 0,
 'Salah_Hissou': 2,
 'Aleksander_Kwaśniewski': 3,
 'Michael_Collins_(Irish_leader)': 5,
 '1936_Summer_Olympics': 0,
 'Parramatta_Eels': 0,
 'Sean_Dundee': 0,
 'RCD_Espanyol': 0,
 'Dominique_Baratelli': 2,
 'ETA': 2,
 'Bernard_Tapie': 7,
 'France': 3,
 'Fredi_Bobic

In [27]:
# CEAFm_precision
CEAFm_precision = sum(max_evaluation_dict.values()) / ents_data.shape[0]
CEAFm_precision

0.08264924571281214

CEAFm recall

In [26]:
# CEAFm_recall
CEAFm_recall = sum(max_evaluation_dict.values()) / ents_data.shape[0]
CEAFm_recall

0.08264924571281214

CEAFm Fscore

In [24]:
# CEAFm_f1
CEAFm_f1 = (2 * (CEAFm_recall * CEAFm_precision)) / (
        CEAFm_precision + CEAFm_recall)
CEAFm_f1

0.15042240300375467